## Load library from Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

PATH_DIR = '/content/drive/MyDrive/XAI-Anna-Carlos/'

import sys
sys.path.append(PATH_DIR)

import xai_faithfulness_experiments_lib as ff

Mounted at /content/drive


## Load MNIST and load a batch

In [2]:
import torch
import torchvision

batch_size = 64

MNIST_PATH = '/files/'

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST(MNIST_PATH, train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST(MNIST_PATH, train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size, shuffle=True)

examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/train-images-idx3-ubyte.gz to /files/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/train-labels-idx1-ubyte.gz to /files/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/t10k-images-idx3-ubyte.gz to /files/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/t10k-labels-idx1-ubyte.gz to /files/MNIST/raw



## Load model

In [3]:
PATH_PRETRAINED = PATH_DIR + 'mnist-classifier.pth'
network = ff.load_pretrained_model(PATH_PRETRAINED)

## Generation parameters

In [4]:
from tqdm import tqdm

num_curves = 100000
num_samples = 20

image = example_data[0]
label = example_targets[0]

# Random generation

In [7]:
import numpy as np
# These will be the generated attribution distributions
rankings = np.zeros((num_curves, *tuple(image.shape)))
# These will be the corresponding output vs selection level plots
plots = np.zeros((num_curves, num_samples))
# These will be the corresponding output vs selection level plots with the inverse rankings
inverse_plots = np.zeros((num_curves, num_samples))
# These will indicate whether for a given selection level, the examined output is the highest activated
hit_plots = np.zeros((num_curves, num_samples), dtype=bool)
# These will indicate whether for a given selection level, the examined output is the highest activated (for the inverse rankings)
inverse_hit_plots = np.zeros((num_curves, num_samples), dtype=bool)
# Measures
qmeans = np.zeros(num_curves)
qmean_invs = np.zeros(num_curves)
qaucs = np.zeros(num_curves)
qauc_invs = np.zeros(num_curves)
qargmaxs = np.zeros(num_curves)
qargmax_invs = np.zeros(num_curves)

for i in tqdm(range(num_curves)):
  ranking = ff._get_random_ranking_image(image.shape[1:])
  rankings[i] = ranking

  result = ff.get_measures_for_ranking(image, ranking, label, network, with_inverse = True, with_random = False)
  plots[i] = result['output_curve']
  hit_plots[i] = result['is_hit_curve']
  inverse_plots[i] = result['output_curve_inv']
  inverse_hit_plots[i] = result['is_hit_curve_inv']
  qmeans[i] = result['mean']
  qmean_invs[i] = result['mean_inv']
  qaucs[i] = result['auc']
  qauc_invs[i] = result['auc_inv']
  qargmaxs[i] = result['at_first_argmax']
  qargmax_invs[i] = result['at_first_argmax_inv']

100%|██████████| 100000/100000 [03:38<00:00, 457.38it/s]


## Save to Google Drive

In [8]:
np.savez(PATH_DIR + 'random_generated-new.npz', \
         image=image, \
         label=label, \
         rankings=rankings, \
         output_curves=plots, \
         output_curves_inv=inverse_plots, \
         is_hit_curves=hit_plots, \
         is_hit_curves_inv=inverse_hit_plots, \
         qmeans=qmeans, \
         qmean_invs=qmean_invs, \
         qargmaxs=qargmaxs, \
         qargmax_invs=qargmax_invs, \
         qaucs=qaucs, \
         qauc_invs=qauc_invs
         )

# Generation with genetic programming
The algorithm is as follows:


*   Generate an initial population of random rankings
*   For a given number of steps, do:
  *   Compute Q measure for current population
  *   Take top `num_saved` elements according to Q measure. Replace the rest of the population with "children" of randomly selected "parents" from the top `num_saved`

First we declare the function that, given two "parent" $p1$ and $p2$ rankings, generates a "child" $c$ that is a mixture of their "parents". In this case, $c_i$ will be a random element of the set $ \left(p1_{\left[0,i\right]} \bigcup p2_{\left[0,i\right]}\right) \setminus c_{\left[0,i-1\right]} $.

Then we declare a function that trims a population to its top `num_saved` performers and repopulates the rest by crossing them at random.

In [9]:
import random

def spawn_rankings(r1, r2):
  # Generates a new ranking that's a mixture of r1 and r2
  new_r = np.zeros(len(r1), dtype=int)
  # The next position of new_r will be sampled from a pool of candidates
  candidates = []
  # We don't want the same element more than once so we keep track of the ones we use
  used = np.zeros(len(r1),bool)
  for i in range(len(new_r)):
    if not used[r1[i]]:
      used[r1[i]] = True
      candidates.append(r1[i])
    if not used[r2[i]]:
      used[r2[i]] = True
      candidates.append(r2[i])
    #print(candidates)
    selected = candidates.pop(random.randint(0,len(candidates)-1))
    #print('Selected',selected)
    #print(candidates)
    new_r[i] = selected
    #print('New_r',new_r,'\n','-'*20)
  return new_r

# TEST
from numpy.random import default_rng
rng = default_rng()
r1 = rng.permutation(8)
r2 = rng.permutation(8)
print('R1',r1)
print('R2',r2)
r_spawn = spawn_rankings(r1, r2)
print(r_spawn)

def repopulate(population, ranking, num_saved):
  for i in range(num_saved, population.shape[0]):
    parent_indices = rng.permutation(num_saved)[:2]
    parent1 = population[ranking[parent_indices[0]]]
    parent2 = population[ranking[parent_indices[1]]]
    population[ranking[i]] = spawn_rankings(parent1, parent1)

R1 [2 4 6 7 5 0 1 3]
R2 [1 2 3 5 7 6 0 4]
[2 4 1 6 7 5 0 3]


In [10]:
# We also need a helper function to transform a linear ranking into an attribution map to be used to compute the output vs selection level plot.
def get_ranking_image_from_linear_ranking(ranking, desired_shape):
  # We want an image with the same size of the original, but were for each pixel
  # we store its position in the ranking (0 - first element, 1 - last element)
  ranking_image = np.zeros(desired_shape)
  num_features = ranking.size
  for i in range(num_features):
    pos_x = ranking[i] // desired_shape[2]
    pos_y = ranking[i] % desired_shape[2]
    ranking_image[0, pos_x, pos_y] = i/num_features
  return torch.from_numpy(ranking_image)

# TEST
print(get_ranking_image_from_linear_ranking(r_spawn,(1,2,4)))

tensor([[[0.7500, 0.2500, 0.0000, 0.8750],
         [0.1250, 0.6250, 0.3750, 0.5000]]], dtype=torch.float64)


In [ ]:
# Actual generation

In [12]:
num_iterations = 10
fraction_stored = 0.5
pop_size = num_curves // int(num_iterations*fraction_stored)
num_saved = 20
num_stored = int(pop_size*fraction_stored)

num_features = image.detach().numpy().size
rankings = None
population = np.zeros((pop_size, num_features), dtype=int)
print('Generating initial population...')
for i in tqdm(range(pop_size)):
  population[i] = rng.permutation(num_features)

image_shape = np.squeeze(image.detach().numpy()).shape
for i in range(num_iterations):
  iteration_qs = []
  for j in tqdm(range(pop_size)):
    result = ff.get_measures_for_attributions(image, population[j].reshape(image_shape), label, network, with_inverse = False, with_random = False, measures=['mean'])
    plots[i] = result['output_curve']
    hit_plots[i] = result['is_hit_curve']
    qmeans[i] = result['mean']
    iteration_qs.append((qmeans[i], j))
  iteration_qs.sort()
  avg_q = np.mean(list(map(lambda x:x[0], iteration_qs)))
  print(f'{i}/{num_iterations} - Avg. Q {avg_q}')
  repopulate(population, list(map(lambda x:x[1], iteration_qs)), num_saved)
  # The population of curves is saved after each iteration
  #np.savez(PATH_DIR + f'genetic_generated-inv-{i}--.npz', image, population, avg_q)
  if rankings is None: # First iteration
    rankings = population[:num_stored]
  else:
    rankings = np.vstack((rankings, population[:num_stored]))

Generating initial population...


100%|██████████| 20000/20000 [01:02<00:00, 319.13it/s]


0/10 - Avg. Q -0.43770825913622974


100%|██████████| 20000/20000 [01:02<00:00, 318.46it/s]


1/10 - Avg. Q -0.9726177839905024


...

7/10 - Avg. Q -1.073746919631958


100%|██████████| 20000/20000 [00:58<00:00, 342.52it/s]


8/10 - Avg. Q -1.073746919631958


100%|██████████| 20000/20000 [00:59<00:00, 335.96it/s]


9/10 - Avg. Q -1.073746919631958


In [14]:
from tqdm import tqdm

num_curves = rankings.shape[0]
num_samples = 20

# These will be the corresponding output vs selection level plots
plots = np.zeros((num_curves, num_samples))
# These will be the corresponding output vs selection level plots with the inverse rankings
inverse_plots = np.zeros((num_curves, num_samples))
# These will indicate whether for a given selection level, the examined output is the highest activated
hit_plots = np.zeros((num_curves, num_samples), dtype=bool)
# These will indicate whether for a given selection level, the examined output is the highest activated (for the inverse rankings)
inverse_hit_plots = np.zeros((num_curves, num_samples), dtype=bool)
# Measures
qmeans = np.zeros(num_curves)
qmean_invs = np.zeros(num_curves)
qaucs = np.zeros(num_curves)
qauc_invs = np.zeros(num_curves)
qargmaxs = np.zeros(num_curves)
qargmax_invs = np.zeros(num_curves)

image_tensor = image#torch.from_numpy(image)

for i in tqdm(range(num_curves)):
  result = ff.get_measures_for_attributions(image, rankings[i].reshape(image_shape), label, network, with_inverse = True, with_random = False)
  plots[i] = result['output_curve']
  hit_plots[i] = result['is_hit_curve']
  inverse_plots[i] = result['output_curve_inv']
  inverse_hit_plots[i] = result['is_hit_curve_inv']
  qmeans[i] = result['mean']
  qmean_invs[i] = result['mean_inv']
  qaucs[i] = result['auc']
  qauc_invs[i] = result['auc_inv']
  qargmaxs[i] = result['at_first_argmax']
  qargmax_invs[i] = result['at_first_argmax_inv']

  0%|          | 0/100000 [00:00<?, ?it/s]/content/drive/MyDrive/XAI-Anna-Carlos/xai_faithfulness_experiments_lib.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
100%|██████████| 100000/100000 [06:39<00:00, 250.15it/s]


In [15]:
np.savez(PATH_DIR + 'genetic_generated-new.npz', \
         image=image, \
         label=label, \
         rankings=rankings, \
         output_curves=plots, \
         output_curves_inv=inverse_plots, \
         is_hit_curves=hit_plots, \
         is_hit_curves_inv=inverse_hit_plots, \
         qmeans=qmeans, \
         qmean_invs=qmean_invs, \
         qargmaxs=qargmaxs, \
         qargmax_invs=qargmax_invs, \
         qaucs=qaucs, \
         qauc_invs=qauc_invs
         )